# Common Relax Workflow

Author: Edan Bainglass (edan.bainglass@psi.ch)

This notebook demonstrates the use of generic input/output OO-LD schemas for a structure 
geometry optimization (relaxation) workflow, here driven by the AiiDA workflow engine.

For more information on the principles behind this work, visit the following [PREMISE organization
repository](https://github.com/ord-premise/common-workflow-schemas).

Requirements:
- `AiiDA~2.6`
- `OPTIMADE~1`

In [ ]:
from common_workflow_schemas.utils.printers import print_json

# Build input

## Code

### Execution Environment

In [ ]:
from common_workflow_schemas.schemas.code import ExecutionEnvironment

execution_environment = ExecutionEnvironment(
    **{
        "name": "localhost-test",
        "metadata": {
            "hostname": "localhost",
            "description": "Test machine",
            "transport_protocol": "local",
            "scheduler": "slurm",
            "queue": "compute",
            "architecture": "x86_64",
            "os": {
                "name": "Linux",
                "metadata": {
                    "distribution": {
                        "name": "Ubuntu",
                        "version": "24.04.2",
                    }
                },
            },
            "preinstalled": False,
            "path": "/usr/bin/pw.x",
        },
    },
)
ee_schema = execution_environment.model_oo_ld()
# print_json(ee_schema)

### Package

#### Manager

In [ ]:
from common_workflow_schemas.schemas.code import PackageManager

package_manager = PackageManager(
    **{
        "name": "conda",
        "metadata": {
            "channel": "conda-forge",
            "version": "24.7.1",
        },
    }
)
pm_schema = package_manager.model_oo_ld()
# print_json(pm_schema)

#### Create Package

In [ ]:
from common_workflow_schemas.schemas.code import Package

package = Package(
    **{
        "name": "qe",
        "package_manager": package_manager,
        "metadata": {
            "version": "7.2",
        },
    }
)
package_schema = package.model_oo_ld()
# print_json(package_schema)

### Create code

In [ ]:
from common_workflow_schemas.schemas.code import Code

code = Code(
    identifier="07f316b1-5403-40eb-b4dc-6be4a529ce67",  # pw-7.4@localhost
    name="Quantum ESPRESSO",
    package=package,
    executionEnvironment=execution_environment,
)
code_schema = code.model_oo_ld()
# print_json(code_schema)

## Engine

In [ ]:
from common_workflow_schemas.schemas.engine import Engine

engine = Engine(
    code=code,
    options={
        "resources": {
            "num_machines": 1,
        },
    },
)
engine_schema = engine.model_oo_ld()
# print_json(engine_schema)

## Relax inputs

### Common inputs

In [ ]:
from common_workflow_schemas.schemas.relax import CommonRelaxInputs

common_inputs = CommonRelaxInputs(
    engines={
        "relax": engine,
    },
    protocol="fast",
    relax_type="positions_cell",
    reference_process="e03d0b01-5ab4-4628-8049-ca2b940ce19a",
)
inputs_schema = common_inputs.model_oo_ld()
# print_json(inputs_schema)

### OPTIMADE Structure

In [ ]:
from optimade.client import OptimadeClient

url = "https://aiida.materialscloud.org/mc3d/optimade"
query = 'elements HAS "Si" AND nsites < 2'
client = OptimadeClient(url)

In [ ]:
%%capture

_ = client.get(query)

In [ ]:
results = client.all_results["structures"][query][url]
structure_entry = results.data[0]

### Create relax inputs

In [ ]:
from common_workflow_schemas.schemas.relax import RelaxInputs

relax_inputs = RelaxInputs(
    structure=structure_entry,
    **common_inputs.model_dump(),
)
ri_schema = relax_inputs.model_oo_ld()
# print_json(ri_schema)

# AiiDA input prep

In [ ]:
from aiida import load_profile, orm

load_profile();

## Expand/compact

In [ ]:
from pyld import jsonld

expanded = jsonld.expand(ri_schema, ri_schema["@context"])  # type: ignore
# print_json(expanded)

In [ ]:
ri_context: dict = ri_schema["@context"]
aiida_context = {
    "@context": {
        key: value  # if key in ("@vocab", "ex", "cw") else f"aiida_{key}": value
        for key, value in ri_context.items()
    }
}

compacted: dict = jsonld.compact(expanded, aiida_context)  # type: ignore
# print_json(compacted["structure"])

## Structure

The following post-processing of the compacted structure data is only
necessary due to the lack of semantic annotation in OPTIMADE, particularly
its `StructureResource` model.

Additionally, the conversion from `StructureResource` to `StructureData` could be
automated via semantics-aware serializers, or dedicated tools, e.g. <a href="https://sintef.github.io/dlite/">dlite</a>


In [ ]:
import numpy as np


def process_structure_resource(structure_data: dict):
    attributes = structure_data["attributes"]

    # These need to be lists
    for key in (
        "elements",
        "elements_ratios",
        "cartesian_site_positions",
        "species",
        "species_at_sites",
    ):
        attributes[key] = [attributes[key]]

    # These need to be lists
    for key in (
        "chemical_symbols",
        "concentration",
        "mass",
    ):
        attributes["species"][0][key] = [attributes["species"][0][key]]

    # This needs to be structure as a 3D vector list
    attributes["lattice_vectors"] = (
        np.array(attributes["lattice_vectors"]).reshape((3, 3)).tolist()
    )


process_structure_resource(compacted["structure"])

In [ ]:
from optimade.adapters.structures.aiida import get_aiida_structure_data
from optimade.models.structures import StructureResource

structure_resource = StructureResource(**compacted["structure"])
structure: orm.StructureData = get_aiida_structure_data(structure_resource)

## Code

Ideally, the provided code information (inspect by running the code below) **should be
made sufficient** for AiiDA to create the necessary nodes (`Computer`, `Code`) for the
requested executable. For example:

```python
> compacted["engines"]["relax"]["code"]
{
    "name": "Quantum ESPRESSO",
    "package": {
        "metadata": {"version": "7.2"},
        "name": "qe",
        "package_manager": {
            "metadata": {
                "channel": "conda-forge",
                "version": "24.7.1",
            },
            "name": "conda",
        },
    },
    "executionEnvironment": {
        "metadata": {
            "architecture": "x86_64",
            "description": "Test machine",
            "hostname": "localhost",
            "os": {
                "metadata": {
                    "distribution": {
                        "name": "Ubuntu",
                        "version": "24.04.2",
                    }
                },
                "name": "Linux",
            },
            "path": "/usr/bin/pw.x",
            "preinstalled": False,
            "queue": "compute",
            "scheduler": "slurm",
            "transport_protocol": "local",
        },
        "name": "localhost-test",
    },
}
```

This is, however, outside the scope of the present phase. Here, we load an existing code.

In [ ]:
code = orm.load_code(compacted["engines"]["relax"]["code"]["identifier"])

In [ ]:
compacted["engines"]["relax"]["code"] = code

## Build input

In [ ]:
acwf_input = {
    "structure": structure,
    "engines": compacted["engines"],
    "protocol": compacted["protocol"],
    "relax_type": compacted["relax_type"],
}


if electronic_type := compacted.get("electronic_type"):
    acwf_input["electronic_type"] = electronic_type

if spin_type := compacted.get("spin_type"):
    acwf_input["spin_type"] = spin_type

if threshold_forces := compacted.get("threshold_forces"):
    acwf_input["threshold_forces"] = threshold_forces

if threshold_stress := compacted.get("threshold_stress"):
    acwf_input["threshold_stress"] = threshold_stress

if reference_process := compacted.get("reference_process"):
    # A similar comment here w.r.t automation as made above for the structure
    acwf_input["reference_workchain"] = orm.load_node(reference_process)

# acwf_inputs

## Submit calculation

In [ ]:
from aiida.engine import submit
from aiida_common_workflows.workflows.relax.quantum_espresso import (
    QuantumEspressoCommonRelaxWorkChain as RelaxWorkChain,
)

builder = RelaxWorkChain.get_input_generator().get_builder(**acwf_input)
submit(builder)

# Output

Here we manually convert AiiDA's outputs to the generic schema. However, in principle,
the above procedure using JSON-LD expansion/compaction on an output OO-LD document, along
with automated data conversion (e.g. using `dlite`), could be used to automate the procedure.

In [ ]:
node = orm.load_node(847)

In [ ]:
from optimade.adapters.structures import Structure

def process(node):
    if isinstance(node, orm.ArrayData):
        return node.get_array()
    if isinstance(node, orm.StructureData):
        return Structure.ingest_from(structure.get_ase(), format="ase").entry
    return node.value

In [ ]:
outputs = {key: process(node.outputs[key]) for key in  node.outputs}

In [ ]:
from common_workflow_schemas.schemas.relax import RelaxOutputs

outputs_model = RelaxOutputs(**outputs)
outputs_model.model_dump()